# Задание

Набор данных ex9_movies.mat представляет собой файл формата *.mat (т.е. сохраненного из Matlab). Набор содержит две матрицы Y и R - рейтинг 1682 фильмов среди 943 пользователей. Значение Rij может быть равно 0 или 1 в зависимости от того оценил ли пользователь j фильм i. Матрица Y содержит числа от 1 до 5 - оценки в баллах пользователей, выставленные фильмам.


1. Загрузите данные ex9_movies.mat из файла.
2. Выберите число признаков фильмов (n) для реализации алгоритма коллаборативной фильтрации.
3. Реализуйте функцию стоимости для алгоритма.
4. Реализуйте функцию вычисления градиентов. При реализации используйте векторизацию для ускорения процесса обучения.
6. Добавьте L2-регуляризацию в модель.
7. Обучите модель с помощью градиентного спуска или других методов оптимизации.
8. Добавьте несколько оценок фильмов от себя. Файл movie_ids.txt содержит индексы каждого из фильмов.
9. Сделайте рекомендации для себя. Совпали ли они с реальностью?
10. Также обучите модель с помощью сингулярного разложения матриц. Отличаются ли полученные результаты?

In [10]:
import math
import numpy as np

from scipy.io import loadmat
from scipy.optimize import minimize, check_grad

%matplotlib inline

1. Загрузите данные ex9_movies.mat из файла.

In [3]:
data = loadmat('Data/Lab 9/ex9_movies.mat')
Y, R = data['Y'], data['R']

Y.shape, R.shape

((1682, 943), (1682, 943))

In [4]:
nm, nu = Y.shape

2. Выберите число признаков фильмов (n) для реализации алгоритма коллаборативной фильтрации.

In [5]:
n = 16

3. Реализуйте функцию стоимости для алгоритма.

Функция стоимости: 
$$ J = \frac{1}{2}(||R * Q - Y||_{2} + \lambda*||Q||_2) = \frac{1}{2}(||R * U^TM - Y||_{2} + \lambda(||U||_2 + ||M||_2))$$

$Q$ - совмещенная матрица пользователей и объектов (фильмов)  
$U$ - матрица скрытых представлений пользователей  
$M$ - матрица скрытых представлений фильмов  
$Y$ - матрица оценок пользователя  

In [6]:
def loss(q, Y, R, nu, nm, k, lam):
    Q = q.reshape((nu+nm, k))
    M, U = Q[:nm], Q[-nu:]
    error = R * M.dot(U.T) - Y
    
    return 0.5 * (np.sum(error**2) + lam * np.sum(Q**2))    

4. Реализуйте функцию вычисления градиентов. При реализации используйте векторизацию для ускорения процесса обучения.

In [7]:
def grad(q, Y, R, nu, nm, k, lam):
    Q = q.reshape((nu+nm, k))
    M, U = Q[:nm], Q[-nu:]
    
    # Chain rule here
    E = M.dot(U.T) - Y
    E_ = R * E
    
    dQ = np.zeros((nu+nm, k))
    dQ[:nm] = E_.dot(U) + lam*M
    dQ[-nu:] = E_.T.dot(M) + lam*U
    return dQ.flatten()

7. Обучите модель с помощью градиентного спуска или других методов оптимизации.

In [12]:
# Начальное приближение
q0 = np.random.rand(nu+nm, n).flatten()

In [13]:
result = minimize(loss, q0, (Y, R, nu, nm, n, 0.1), jac=grad, method="L-BFGS-B")
result

      fun: 17322.38738488291
 hess_inv: <42000x42000 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00118572,  0.00121669, -0.00030646, ..., -0.0002981 ,
       -0.00096843,  0.00139672])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 7109
      nit: 6888
   status: 0
  success: True
        x: array([0.46351324, 0.30475258, 0.31983096, ..., 1.1791122 , 0.86168118,
       0.52031682])

In [40]:
Q = result.x.reshape((nu+nm, n))
M, U = Q[:nm], Q[-nu:]

Y[1, 0], np.sum(M[1] * U[0])

(3, 3.0077188342420307)

8. Добавьте несколько оценок фильмов от себя. Файл movie_ids.txt содержит индексы каждого из фильмов.

In [42]:
forward_dict, backward_dict = {}, {}

with open("Data/Lab 9/movie_ids.txt", "rb") as movies_file:
    for line in movies_file:
        try:
            line = line.decode("utf-8")
            movie_id, movie_name = line.split(maxsplit=1)
            movie_id = int(movie_id)
            
            forward_dict[movie_id] = movie_name.strip()
            backward_dict[movie_name] = movie_id
        except:
            pass

1673

In [45]:
# 23 = Taxi Driver (1976), 127 = Godfather, The (1972)
my_vector = np.zeros(Y.shape[0])
my_vector[23 - 1] = 5
my_vector[127 - 1] = 5

my_view_vector = np.zeros(Y.shape[0])
my_view_vector[23 - 1] = 1
my_view_vector[127 - 1] = 1

In [59]:
Y_new = np.concatenate((Y, np.expand_dims(my_vector, axis=1)), axis=1)
R_new = np.concatenate((R, np.expand_dims(my_view_vector, axis=1)), axis=1)

np.nonzero(Y_new[:, 943]), np.nonzero(R_new[:, 943])

((array([ 22, 126]),), (array([ 22, 126]),))

9. Сделайте рекомендации для себя. Совпали ли они с реальностью?

In [60]:
nm, nu = Y_new.shape

In [62]:
q0 = np.random.rand(nu+nm, n).flatten()

result = minimize(loss, q0, (Y_new, R_new, nu, nm, n, 0.1), jac=grad, method="L-BFGS-B")
result

      fun: 17350.716102039198
 hess_inv: <42016x42016 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.09144061,  0.10567593,  0.11027979, ..., -0.00016531,
       -0.00048208, -0.00022807])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 11633
      nit: 11290
   status: 0
  success: True
        x: array([0.29671598, 0.65770161, 0.41276589, ..., 0.88429911, 0.36940694,
       0.47325331])

In [64]:
Q = result.x.reshape((nu+nm, n))
M, U = Q[:nm], Q[-nu:]

In [68]:
my_hidden_vector = U[-1, :]

In [82]:
recomendations = M.dot(my_hidden_vector)

for idx in np.where(recomendations >= 4.5)[0]:
    print(forward_dict[idx + 1])

Taxi Driver (1976)
Silence of the Lambs, The (1991)
Godfather, The (1972)
2001: A Space Odyssey (1968)
Apocalypse Now (1979)
GoodFellas (1990)
Psycho (1960)
Godfather: Part II, The (1974)
Raging Bull (1980)
Paradise Lost: The Child Murders at Robin Hood Hills (1996)
One Flew Over the Cuckoo's Nest (1975)
Short Cuts (1993)
For Whom the Bell Tolls (1943)
Tin Drum, The (Blechtrommel, Die) (1979)
High Noon (1952)
Primary Colors (1998)
Living in Oblivion (1995)
Safe (1995)
When We Were Kings (1996)
Welcome To Sarajevo (1997)
Top Hat (1935)
Mina Tannenbaum (1994)
Pather Panchali (1955)
Boys, Les (1997)
World of Apu, The (Apur Sansar) (1959)
Angel Baby (1995)
